In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms, models
import torch.optim as optim
import os
import random
import cv2
import numpy as np
import glob
from PIL import Image


from deformable_conv import DeformableConv2d

In [2]:
'''
Makeup_Cosmetic - Makeup_Co_5
Makeup_Impersonation - Makeup_Im_1
Makeup_Obfuscation - Makeup_Ob_1
Mannequin - Mask_Mann_1
Mask_HalfMask - Mask_Half_1
Mask_PaperMask - Mask_Paper_1
Mask_TransparentMask - Mask_Trans_51
Paper - Paper_1
Partial_Eye - Partial_Eye_3
Partial_FunnyeyeGlasses - Partial_Funnyeye_1
Partial_Mouth - Partial_Mouth_1
Partial_PaperGlasses - Partial_Paperglass_1
Replay - Replay_2
Silicone - Mask_Silicone_4
'''

attack_to_cls = {
    'Makeup_Co': 1,
    'Makeup_Im': 2,
    'Makeup_Ob': 3,
    'Mask_Mann': 4,
    'Mask_Half': 5,
    'Mask_Paper': 6,
    'Mask_Trans': 7,
    'Paper': 8,
    'Partial_Eye': 9,
    'Partial_Funnyeye': 10,
    'Partial_Mouth': 11,
    'Partial_Paperglass': 12,
    'Replay': 13,
    'Mask_Silicone': 14
}

In [3]:
class SiWMv2Dataset(Dataset):
    def __init__(self, live_path, spoof_path, live_ref, spoof_ref):
        self.transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(),transforms.Normalize(mean=[0.5], std=[0.5])])
        self.live_path = live_path
        self.spoof_path = spoof_path
        self.images = []
        self.labels = []
        self.sub_levels = []
        self.real = 0
        self.attack = 1
        
        with open(live_ref, 'r') as f:
            content = f.read()
        live_dirs = content.split('\n')
        
        with open(spoof_ref, 'r') as f:
            content = f.read()
        spoof_dirs = content.split('\n')
        
        for ldir in live_dirs:
            if ldir != '' and ldir != ' ':
                ldir_path = os.path.join(self.live_path, ldir)
                if os.path.exists(ldir_path):
                    lcontent = os.listdir(ldir_path)
                    lcontent = [os.path.join(ldir_path, f) for f in lcontent if f.endswith('png')]
                    self.images.extend(lcontent)
                    self.labels.extend([self.real] * len(lcontent))
                    self.sub_levels.extend([self.real] * len(lcontent))
                else:
                    print(ldir_path)
        
        for sdir in spoof_dirs:
            if sdir != '' and sdir != ' ':
                sdir_path = os.path.join(self.spoof_path, sdir)
                scontent = os.listdir(sdir_path)
                scontent = [os.path.join(sdir_path, f) for f in scontent if f.endswith('png')]
                self.images.extend(scontent)
                self.labels.extend([self.attack] * len(scontent))
                attacklabel = attack_to_cls['_'.join(os.path.basename(sdir).split('_')[:-1])]
                self.sub_levels.extend([attacklabel] * len(scontent))
        
        self.data_len = len(self.images)
        

    def __len__(self): # returns the total number of samples in the dataset
        return self.data_len

    def __getitem__(self, idx): # loads and returns a sample from the dataset at the given index
        imgpath = self.images[idx]
        img = Image.open(imgpath).convert('RGB')
        img = self.transform(img)
        return img, self.labels[idx], self.sub_levels[idx]

In [4]:
device = torch.device(0)

class_label_real = 0
class_label_attack = 1

real_dir = r'/data/ssd2/shakeel-workspace/DOWNLOAD/SiW-Mv2_preprocessed/Live/'
spoof_dir = r'/data/ssd2/shakeel-workspace/DOWNLOAD/SiW-Mv2_preprocessed/Spoof_root/'

# Protocol 1

train_spoof_ref = r'/data/ssd2/shakeel-workspace/DOWNLOAD/SiW-Mv2_preprocessed/trainlist_all.txt'
train_live_ref = r'/data/ssd2/shakeel-workspace/DOWNLOAD/SiW-Mv2_preprocessed/trainlist_live.txt'
test_spoof_ref = r'/data/ssd2/shakeel-workspace/DOWNLOAD/SiW-Mv2_preprocessed/testlist_all.txt'
test_live_ref =r'/data/ssd2/shakeel-workspace/DOWNLOAD/SiW-Mv2_preprocessed/testlist_live.txt'

# train_spoof_ref = r'protocol_1/train_spoof.txt'
# train_live_ref = r'protocol_1/train_live.txt'
# val_spoof_ref = r'protocol_1/val_spoof.txt'
# val_live_ref = r'protocol_1/val_live.txt'
# test_spoof_ref = r'/data/ssd2/shakeel-workspace/DOWNLOAD/SiW-Mv2_preprocessed/testlist_all.txt'
# test_live_ref =r'/data/ssd2/shakeel-workspace/DOWNLOAD/SiW-Mv2_preprocessed/testlist_live.txt'

# train_dataset = SiWMv2Dataset(real_dir, spoof_dir, train_live_ref, train_spoof_ref)
# val_dataset = SiWMv2Dataset(real_dir, spoof_dir, val_live_ref, val_spoof_ref)
test_dataset = SiWMv2Dataset(real_dir, spoof_dir, test_live_ref, test_spoof_ref)

# train_loader = DataLoader(train_dataset, batch_size  = 64, shuffle = True, pin_memory = True, num_workers = 8)
# val_loader = DataLoader(val_dataset, batch_size  = 64, shuffle = True, pin_memory = True, num_workers = 8)
test_loader = DataLoader(test_dataset, batch_size  = 64, shuffle = True, pin_memory = True, num_workers = 8)

# Print dataset sizes
print(f"Training set size: {len(test_dataset)}")
# print(f"Training set size: {len(val_dataset)}")

/data/ssd2/shakeel-workspace/DOWNLOAD/SiW-Mv2_preprocessed/Live/Live_893
/data/ssd2/shakeel-workspace/DOWNLOAD/SiW-Mv2_preprocessed/Live/Live_896
/data/ssd2/shakeel-workspace/DOWNLOAD/SiW-Mv2_preprocessed/Live/Live_899
/data/ssd2/shakeel-workspace/DOWNLOAD/SiW-Mv2_preprocessed/Live/Live_900
Training set size: 110019


In [5]:
# Load pre-trained ResNet18
# model = models.resnet18(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 2)

# Load pre-trained MobileNetV2
model = models.mobilenet_v2(pretrained=True)
# model.features[0] = DeformableConv2d(3, 32, 3, 2, 1)
model.features[-1] = nn.Sequential(
    DeformableConv2d(320, 1280, 3, 2),
    nn.BatchNorm2d(1280),
    nn.ReLU6()
)
model.classifier[1] = nn.Linear(in_features=1280, out_features=2) #default in_features =1280, out_features = 1000

# best_model_path = r'checkpoint_protocol_1_wo_val/best_95_0.000376204145941474.pth'
# best_model_path = r'checkpoint_protocol_1_wo_val/best_105_0.00012885049953524681.pth'
# best_model_path = r'checkpoint_protocol_1_wo_val/best_126_0.00012175165403143548.pth'
best_model_path = r'checkpoint_protocol_1_wo_val/best_142_4.0373697962606216e-07.pth'  #3.009
# best_model_path = r'checkpoint_protocol_1_wo_val_cont/best_2_0.0005171791127407008.pth'
# best_model_path = r'checkpoint_protocol_1_wo_val/best_32_0.0007601213937333313.pth'
# best_model_path = r'checkpoint_protocol_1_wo_val/best_135_0.00016639192170754796.pth'
# best_model_path = r'checkpoint_protocol_1_wo_val_cont/best_127_2.6473109944895285e-05.pth'
# best_model_path = r'checkpoint_protocol_1_wo_val_cont/best_188_5.549576379481903e-05.pth'
state_dict = torch.load(best_model_path,  map_location=device)
model.load_state_dict(state_dict, strict = True)
model = model.to(device)
print(model)

/home/eslab/anaconda3/envs/facerec_new/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/eslab/anaconda3/envs/facerec_new/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [6]:
# Evaluate on the test set
test_correct = 0
test_total = 0

model.eval()
with torch.no_grad():
    
    test_cat_labels = torch.empty(0, dtype=torch.int64, device=device)
    test_cat_labels_sub = torch.empty(0, dtype=torch.int64, device = device)
    test_predicted_cat_labels = torch.empty(0, dtype=torch.int64, device=device)

    for test_images, test_labels, test_labels_sub in test_loader:
        test_images, test_labels = test_images.to(device), test_labels.to(device)
        test_labels_sub = test_labels_sub.to(device)
        test_model_op = model(test_images)
        _, test_predicted = torch.max(test_model_op, 1)
        test_correct += (test_predicted == test_labels).sum().item() 
        test_total += test_labels.size(0)

        test_cat_labels = torch.cat((test_cat_labels, test_labels))
        test_cat_labels_sub = torch.cat((test_cat_labels_sub, test_labels_sub))
        test_predicted_cat_labels = torch.cat((test_predicted_cat_labels, test_predicted))

    test_accuracy = test_correct / test_total * 100  
    print(f'Test Accuracy: {test_accuracy:.2f}%')

Test Accuracy: 95.16%


In [7]:
test_cat_labels_cpu = test_cat_labels.cpu()
test_cat_labels_sub = test_cat_labels_sub.cpu()
test_predicted_cat_labels_cpu = test_predicted_cat_labels.cpu()

In [8]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score

In [9]:
tn, fp, fn, tp = confusion_matrix(test_cat_labels_cpu, test_predicted_cat_labels_cpu).ravel()

print(f'TN: {tn}, FP: {fp}, FN: {fn}, TP: {tp}')

acc_score = accuracy_score(test_cat_labels_cpu, test_predicted_cat_labels_cpu)
prec_score = precision_score(test_cat_labels_cpu, test_predicted_cat_labels_cpu)
recall = recall_score(test_cat_labels_cpu, test_predicted_cat_labels_cpu)

Y_I_val =(tp/(tp+fn)) + (tn/(tn+fp)) - 1
sensitivity_val = tp / (tp + fn)
specificity_val = tn / (tn + fp)
f1score_val = 2 * tp / (2 * tp + fp + fn)
FAR = fp/(fp + tn)
FRR = fn/(fn + tp)
HTER_val = (FAR + FRR)/2
EER = (fp+fn)/(tn+fp+fn+tp)
val_bacc = balanced_accuracy_score(test_cat_labels_cpu, test_predicted_cat_labels_cpu)


print('Testing Results')
print(30*'-')
print('Acc:', acc_score, '\nSen:', sensitivity_val, '\nSpec:', specificity_val, '\nYI:', Y_I_val, '\nF1:', f1score_val, '\nPrec:', prec_score, '\nRecall:', recall, '\nHTER:', HTER_val, '\nEER:', EER, '\nBACC:', val_bacc)

TN: 55137, FP: 575, FN: 4749, TP: 49558
Testing Results
------------------------------
Acc: 0.9516083585562494 
Sen: 0.9125527095954481 
Spec: 0.9896790637564618 
YI: 0.9022317733519098 
F1: 0.949023362696285 
Prec: 0.9885305088464684 
Recall: 0.9125527095954481 
HTER: 0.04888411332404505 
EER: 0.048391641443750626 
BACC: 0.9511158866759549


In [10]:
from siw_metrics import my_metrics

In [21]:
apcer_lst = []
bpcer_lst = []
acer_lst = []

In [12]:
print(test_predicted_cat_labels_cpu)
print(test_cat_labels_sub)
print(test_cat_labels_cpu)

tensor([1, 1, 1,  ..., 0, 1, 0])
tensor([13,  9, 10,  ...,  0,  8,  0])
tensor([1, 1, 1,  ..., 0, 1, 0])


In [13]:
real_labels = [index for index, element in enumerate(test_cat_labels_sub) if element == 0]

In [14]:
unique_vals = torch.unique(test_cat_labels_sub)
print(unique_vals)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])


In [15]:
unique_vals = unique_vals[1:]
print(unique_vals)

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])


In [23]:
label_current

array(1)

In [24]:
for i  in unique_vals:
    attack_indices = [index for index, element in enumerate(test_cat_labels_sub) if element == i.item()]
    pred_current = [test_predicted_cat_labels_cpu[i].item() for i in attack_indices]
    # label_current = [ayo[i] for i in attack_indices]
    label_current = [test_cat_labels_cpu[i].item() for i in attack_indices]
    for j in real_labels:
        label_current.append(test_cat_labels_cpu[j])
        pred_current.append(test_predicted_cat_labels_cpu[j])
    # label_current = np.where(label_current == 1, 0, 1)
    # label_current = torch.Tensor(label_current)
    
    label_current = np.where(np.array(label_current) == 1, 0, 1)
    pred_current = np.where(np.array(pred_current) == 1, 0, 1)
    
    best_AP, best_BP, best_ACER, EER, res_tpr, auc_score  = my_metrics(label_current, pred_current, val_phase=True)
    apcer_lst.append(best_AP)
    bpcer_lst.append(best_BP)
    acer_lst.append(best_ACER)

In [25]:
print(apcer_lst)
print(bpcer_lst)
print(acer_lst)

[0.06876729827549499, 0.0005359056806002144, 0.0006531678641410843, 0.08571428571428572, 0.0, 0.001388888888888889, 0.6470369682333271, 0.07812235214370446, 0.005878510777269758, 0.0011458333333333333, 0.0, 0.0, 0.029807493272614366, 0.06908874329958309]
[0.010320936243538248, 0.010320936243538248, 0.010320936243538248, 0.010320936243538248, 0.010320936243538248, 0.010320936243538248, 0.010320936243538248, 0.010320936243538248, 0.010320936243538248, 0.010320936243538248, 0.010320936243538248, 0.010320936243538248, 0.010320936243538248, 0.010320936243538248]
[0.03954411725951662, 0.005428420962069231, 0.005487052053839666, 0.04801761097891198, 0.005160468121769124, 0.0058549125662135685, 0.3286789522384327, 0.04422164419362135, 0.008099723510404003, 0.005733384788435791, 0.005160468121769124, 0.005160468121769124, 0.020064214758076307, 0.03970483977156067]


In [26]:
max(apcer_lst)

0.6470369682333271

In [27]:
max(bpcer_lst)

0.010320936243538248

In [28]:
max(acer_lst)

0.3286789522384327

In [29]:
(0.6470369682333271 + 0.010320936243538248) * 0.5

0.3286789522384327